# 12.1. Предобработка большого датасета CARLA (seq5)

## Предобработка большого датасета CARLA

In [1]:
%config Completer.use_jedi = False

%load_ext autoreload
%autoreload 2


import pickle

from itertools import islice

from tqdm.auto import tqdm

from registration_metrics import (
    CARLA_dataset,
    visualize_clds,
    transform_pcd,
)

from paths import ROOT_DIR

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
%%time

data = CARLA_dataset(
    dir_path=None,
    filename=f"{ROOT_DIR}/carla_semantic/seq5_withlabels/word_cords.csv",
    start_id=1000,
    end_id=1030,
    dataset_coords_type="world",
    wanted_coords_type="frame"
)

CPU times: user 9.9 s, sys: 4.76 s, total: 14.7 s
Wall time: 26.1 s


In [4]:
%%time

data = CARLA_dataset(
    dir_path=None,
    filename=f"{ROOT_DIR}/carla_semantic/seq5_withlabels/word_cords.csv",
    start_id=0,
    end_id=30,
    dataset_coords_type="world",
    wanted_coords_type="frame"
)

CPU times: user 6.28 s, sys: 2.87 s, total: 9.15 s
Wall time: 3.41 s


Считаем все облака из CSV файла:

In [144]:
filename=f"{ROOT_DIR}/carla_semantic/seq5_withlabels/word_cords.csv"

seq5 = []
with open(filename, mode='r') as csv_file:
    for line_number, elem in enumerate(tqdm(csv_file)):
        frame = dict(CARLA_dataset.parse_line(elem))
        seq5.append(frame)

In [55]:
tmps = pickle.dumps(seq5)
len(tmps) / 2 ** 20

2147.4817323684692

Сохраним считанные облака из CSV файла в отдельные файлы:

In [71]:
for pc_idx, frame in enumerate(tqdm(seq5)):
    with open(f"{ROOT_DIR}/carla_semantic/seq5_withlabels/"
              f"point_clouds/{pc_idx:06}.bin", "wb") as fout:
        pickle.dump(frame, fout)

  0%|          | 0/2422 [00:00<?, ?it/s]

Проверка записанных облаков:

In [122]:
with open(f"{ROOT_DIR}/carla_semantic/seq5_withlabels/point_clouds/"
          f"{0:06}.bin", "rb") as fin:
    frame = pickle.load(fin)

Проверка скорости работы старого и нового интерфейсов:

In [5]:
%%time

data_old = CARLA_dataset(
    dir_path=None,
    filename=f"{ROOT_DIR}/carla_semantic/seq5_withlabels/word_cords.csv",
    start_id=0,
    end_id=100,
    dataset_coords_type="world",
    wanted_coords_type="frame"
)

CPU times: user 21 s, sys: 9.11 s, total: 30.1 s
Wall time: 10.7 s


In [6]:
%%time

data = CARLA_dataset(start_id=0, end_id=100, dataset_coords_type="world")

CPU times: user 940 ms, sys: 747 ms, total: 1.69 s
Wall time: 559 ms


## Работа с большим датасетом CARLA

In [23]:
data = CARLA_dataset(
    dir_path=None,
    filename=f"{ROOT_DIR}/carla_semantic/seq5_withlabels/word_cords.csv",
    start_id=0,
    end_id=30,
    dataset_coords_type="world",
    wanted_coords_type="frame"
)

In [24]:
clds_big = data.clds

In [17]:
clds = [
    transform_pcd(cld, T)
    for cld, T in islice(zip(data.clds, data.poses), None, None, 29)
]

visualize_clds(
    clds + data.clds[::29],
    label_names=["s5f0 world coords", "s5f30 world coords",
                 "s5f0 frame coords", "s5f30 frame coords"],
    save_as="CARLA-seq5-big--visualization", draw=False
)

In [3]:
data = CARLA_dataset(
    dir_path=None,
    filename=f"{ROOT_DIR}/carla_semantic/seq5_withlabels/orig.csv",
    start_id=0,
    end_id=30,
    dataset_coords_type="frame",
    wanted_coords_type="frame"
)

In [26]:
clds_small = data.clds

In [19]:
clds = [
    transform_pcd(cld, T)
    for cld, T in islice(zip(data.clds, data.poses), None, None, 29)
]

visualize_clds(
    clds + data.clds[::29],
    label_names=["s5f0 world coords", "s5f30 world coords",
                 "s5f0 frame coords", "s5f30 frame coords"],
    save_as="CARLA-seq5-small--visualization", draw=False
)